__This Santander value prediction challenge is kind of ruined after a leak is found by some really smart Kagglers. However, one can still learn skills from the procedure of searching for leaks.__

In [1]:
import pandas as pd
import numpy as np

In [7]:
test = pd.read_csv('./test.csv')
test.shape

(49342, 4992)

The initially found leak is shown below.
All data are found to be both time series along rows and columns. Target values are actually two time steps ahead of column 'f190486d6'. Whenever the target should be zero according to this leak rule, the row is removed. Columns and rows are randomly distributed in 4991 columns and 53801 rows.
There are some features of this leak:
1. It generally resembles a Toeplitz matrix.
2. There is a strange numbers occupying the upper right part of the matrix.

In [2]:
# Columns for the initially found leak
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
        'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 
        'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']

# Get rows for the initially found leak
pattern_116 = pd.read_csv('./new_leak_patterns/pattern_1166666.66.csv')
rows = list(pattern_116['Unnamed: 0'])

# Read train data
train = pd.read_csv('./train.csv')

pd.set_option('display.max_columns',41)
train.loc[rows,['target']+cols]

,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
1757,115636.36,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,834800.00,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,296444.44,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,247166.66,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,550000.00,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,3076666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,440000.00,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,1600000.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,15

### 1. Search for leak by locating wired numbers

In [3]:
# Explore the possibility of expending leak columns
train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')
# train = train.drop(['target'],axis=1)
# train = pd.concat([train,test],axis=0)
# train = train.reset_index(drop=True)
# Count times of appearance of the wired number in each column
count = []
for column in train.columns:
    count.append(sum(train[column] == 1166666.66))

d = {'A':train.columns,'B':count}
helper_df = pd.DataFrame(d)
# Pick out columns which contain the wired number and sort them in an ascending order of appearance times
helper_df = helper_df.sort_values(['B'])
helper_df.loc[helper_df['B'] > 0].T

,3147,190,1,3930,4245,2245,3219,4242,551,2858,1582,500,2035,2121,3313,657,1425,3032,4147,3088,...,4045,3708,4023,4476,1566,40,871,2344,728,4446,1484,2760,2934,4859,2157,3575,1974,346,2398,597
A,8e4d0fe45,beb7f98fd,target,d6bb78916,15ace8c9f,241f0f867,4824c1e90,62e59a501,6c0e0801a,02861e414,aac52d8d9,041c5d0c9,d7875bb6c,d48c08bda,e7c0cfd0f,0c9462c08,57dd44c29,a93118262,850027e38,db3839ab0,...,06393096a,13bdd610a,d7d314edc,9a07d7b1f,4d2671746,822e49b95,3c8a3ced0,83635fb67,1857fbccf,c4972742d,b6c0969a2,e78e3031b,36a9a8479,e79e5f72c,092271eb3,74d7f2dc3,277ef93fc,b30e932ba,8f57141ec,350473311
B,1,1,1,1,1,1,1,1,2,3,4,5,6,7,7,8,9,10,11,12,...,18,19,20,21,22,23,24,25,26,27,28,29,30,30,31,32,33,34,35,36


In [4]:
new_cols = helper_df.loc[helper_df['B'] > 0,'A'].values
train.loc[rows,new_cols]

,8e4d0fe45,beb7f98fd,target,d6bb78916,15ace8c9f,241f0f867,4824c1e90,62e59a501,6c0e0801a,02861e414,aac52d8d9,041c5d0c9,d7875bb6c,d48c08bda,e7c0cfd0f,0c9462c08,57dd44c29,a93118262,850027e38,db3839ab0,...,06393096a,13bdd610a,d7d314edc,9a07d7b1f,4d2671746,822e49b95,3c8a3ced0,83635fb67,1857fbccf,c4972742d,b6c0969a2,e78e3031b,36a9a8479,e79e5f72c,092271eb3,74d7f2dc3,277ef93fc,b30e932ba,8f57141ec,350473311
1757,1166666.66,3255483.88,115636.36,1563411.76,1563411.76,1563411.76,3255483.88,1563411.76,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
3809,1830000.00,3255483.88,834800.00,1563411.76,1563411.76,1563411.76,3255483.88,1563411.76,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
511,540000.00,3255483.88,296444.44,1563411.76,1563411.76,1563411.76,3255483.88,1563411.76,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
3798,150666.66,3255483.88,247166.66,1563411.76,1563411.76,1563411.76,3255483.88,1563411.76,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
625,1620000.00,3255483.88,550000.00,1563411.76,1563411.76,1563411.76,3255483.88,1563411.76,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
3303,360800.00,3255483.88,3076666.66,1563411.76,1015000.00,1563411.76,3255483.88,1563411.76,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
4095,440400.00,3255483.88,440000.00,1563411.76,540000.00,1563411.76,3255483.88,1563411.76,360800.00,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
1283,5893333.34,3255483.88,1600000.00,1563411.76,834800.00,1563411.76,3225923.08,1563411.76,809000.00,440400.00,360800.00,1620000.00,150666.66,1830000.00,540000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,...,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66
4209,0.00,3255483.88,466461.54,15

Then, abandon columns like 'beb7f98fd', 'd6bb78916' and '15ace8c9f' that are clearly not right. Adjust orders of some columns that contain the same number of wired number. Find the leading column that contains no wired number but shares all other numbers. A new series of leak columns is found.

### 2. Search for leak in a less constricted but unreliable way 

At first, I hope to discover some new leak that can be used to predict target values. In this way, I can surpass others in the leaderboard. Therefore, I started from values in train target that is not predicted by the original leak columns. This value may just appear in few columns. One of the locations of this value is chosen as a starting point. Following codes will try to search for a Toeplitz matrix around this starting point.

In [168]:
sp = [169,'beb7f98fd'] # The starting point

In [169]:
# All points contain the same value as the starting point are located
possible_points = []
columns = [f for f in train.columns if f not in ['ID','target',sp[1]]]
for column in columns:
    matched_index = list(train.index[train[column] == train.loc[sp[0],sp[1]]])
    for index in matched_index:
        possible_points.append([index,column])

In [170]:
# Since we are searching for a Toeplitz matrix, Once we put three points containing the same value as the starting point
# in the center diagonal line. the diagonal line above and below it should also contain just a constant.
starting_df = [] # Stores a list of 3*3 dataframes which resemble Toeplitz matrix
for i in np.arange(len(possible_points)):
    for j in np.arange(len(possible_points)):
        if j != i:
            ul = possible_points[i] # Upper left point
            lr = possible_points[j] # lower right point
            if (ul[0] == lr[0]) | (ul[0] == sp[0]) | (sp[0] == lr[0]): # Starting, upper left and lower right point should be in totally different columns and rows
                continue
            if train.loc[ul[0],sp[1]] == train.loc[sp[0],lr[1]] and train.loc[sp[0],ul[1]] == train.loc[lr[0],sp[1]]:
                if (train.loc[ul[0],sp[1]] != 0) | (train.loc[sp[0],ul[1]] != 0): # Get rid of situations where zeros appears in the 
                    starting_df.append(train.loc[[ul[0],sp[0],lr[0]],[ul[1],sp[1],lr[1]]])
                    print(len(starting_df))
                    print(train.loc[[ul[0],sp[0],lr[0]],[ul[1],sp[1],lr[1]]])
                    print('\n')

1
       d83a2b684   beb7f98fd  17be6c4e7
2916  6592000.00  1908000.00  5718000.0
169   4240666.66  6592000.00  1908000.0
3153  4240666.66  4240666.66  6592000.0


2
       d83a2b684   beb7f98fd  c7775aabf
2916  6592000.00  1908000.00  5718000.0
169   4240666.66  6592000.00  1908000.0
3153  4240666.66  4240666.66  6592000.0


3
      9884166a7   beb7f98fd   fd99c18b5
470   6592000.0  4240666.66  4240666.66
169   2735000.0  6592000.00  4240666.66
1941  1908000.0  2735000.00  6592000.00


4
       fd99c18b5   beb7f98fd  9884166a7
1941  6592000.00  2735000.00  1908000.0
169   4240666.66  6592000.00  2735000.0
470   4240666.66  4240666.66  6592000.0


5
      17be6c4e7   beb7f98fd   d83a2b684
3153  6592000.0  4240666.66  4240666.66
169   1908000.0  6592000.00  4240666.66
2916  5718000.0  1908000.00  6592000.00


6
      c7775aabf   beb7f98fd   d83a2b684
3153  6592000.0  4240666.66  4240666.66
169   1908000.0  6592000.00  4240666.66
2916  5718000.0  1908000.00  6592000.00




After a 3*3 starting Toepolitz matrix is found. It's possible to use Jiazhen's code to extend the matrix column-wise then row-wise.

In [226]:
train = pd.read_csv('./train.csv')
No = 4
# ----------------------------------------
# Following codes are copied from Jiazhen
rows = list(starting_df[No-1].index)
df_cand_col = train.loc[rows, :]
df_cand_col = df_cand_col.iloc[:, 2:]
cols = list(starting_df[No-1].columns)
df_new = train.loc[rows, cols]

In [227]:
def bf_search(df_new, df_cand, lag):
    cnt = 0
    head_curr = df_new.values[lag:, 0]
    tail_curr = df_new.values[:-lag, -1]
    while True:
        for c in df_cand.columns:
            if c in df_new:
                continue
            elif np.all(
                df_cand[c].iloc[:-lag].values==head_curr
            ) and len(df_cand[c].unique())>1:
                df_new.insert(0, c, df_cand[c].values)
                head_curr = df_new.values[lag:, 0]
                print(c, 'found head!', 'new shape', df_new.shape)
                cnt += 1
                break
            elif np.all(
                df_cand[c].iloc[lag:].values==tail_curr
            ) and len(df_cand[c].unique())>1:
                df_new[c] = df_cand[c].values
                tail_curr = df_new.values[:-lag, -1]
                print(c, 'found tail!', 'new shape', df_new.shape)
                cnt += 1
                break
            else:
                continue
        if cnt==0:
            break
        else:
            cnt = 0
            continue
    return df_new

In [228]:
# Perform column search
print('Column searching ...')
df_new = bf_search(df_new, df_cand_col,1)

Column searching ...
c7775aabf found tail! new shape (3, 4)
74d7998d4 found tail! new shape (3, 5)
9666bfe76 found tail! new shape (3, 6)
a6229abfb found tail! new shape (3, 7)
47665e3ce found tail! new shape (3, 8)
21467a773 found tail! new shape (3, 9)
ee0b53f05 found tail! new shape (3, 10)
025dea3b3 found tail! new shape (3, 11)
c73c31769 found tail! new shape (3, 12)
ef4b87773 found tail! new shape (3, 13)
4f8b27b6b found tail! new shape (3, 14)
90b0ed912 found tail! new shape (3, 15)
d208491c8 found tail! new shape (3, 16)
cf3841208 found tail! new shape (3, 17)
1ec48dbe9 found tail! new shape (3, 18)
01fdd93d3 found tail! new shape (3, 19)
13b54db14 found tail! new shape (3, 20)
55a763d90 found tail! new shape (3, 21)
fa6e76901 found tail! new shape (3, 22)
42fdff3a0 found tail! new shape (3, 23)
ec5764030 found tail! new shape (3, 24)


In [229]:
# Rotate the dataframe and use the same function to perform row search
df_new = df_new.T.copy()
df_cand_row = train[df_new.index].T.copy()
print('Row searching ...')
df_new = bf_search(df_new, df_cand_row,1)

Row searching ...


In [230]:
pd.set_option('display.max_columns',100)
df_new.T

,fd99c18b5,beb7f98fd,9884166a7,c7775aabf,74d7998d4,9666bfe76,a6229abfb,47665e3ce,21467a773,ee0b53f05,025dea3b3,c73c31769,ef4b87773,4f8b27b6b,90b0ed912,d208491c8,cf3841208,1ec48dbe9,01fdd93d3,13b54db14,55a763d90,fa6e76901,42fdff3a0,ec5764030
1941,6592000.00,2735000.00,1908000.0,0.0,5718000.0,688000.0,4454000.0,228000.0,0.0,0.0,714000.0,5446000.0,0.0,0.0,1071000.0,596000.0,6102000.0,3229333.34,698000.00,0.00,0.0,834000.0,0.0,1546000.0
169,4240666.66,6592000.00,2735000.0,1908000.0,0.0,5718000.0,688000.0,4454000.0,228000.0,0.0,0.0,714000.0,5446000.0,0.0,0.0,1071000.0,596000.0,6102000.00,3229333.34,698000.00,0.0,0.0,834000.0,0.0
470,4240666.66,4240666.66,6592000.0,2735000.0,1908000.0,0.0,5718000.0,688000.0,4454000.0,228000.0,0.0,0.0,714000.0,5446000.0,0.0,0.0,1071000.0,596000.00,6102000.00,3229333.34,698000.0,0.0,0.0,834000.0


Since some rows are removed because their target should be zero following the leak rule, it's better to do a row search with a lag of two columns.

In [231]:
print('Row searching ...')
df_new = bf_search(df_new, df_cand_row, 2)

Row searching ...


Since data contains lots of zeros and other repeated numbers, this method doesn't work so well.

### 3. Search for leak in a more constricted and reliable way 

It seems that only the initial leak found decides target values. Later time-series columns can only be used as verifications. So if several rows form time series upon initially found leak columns, they should also form time series upon new leak columns for which we are searching.

In [23]:
# Read train data
train = pd.read_csv('./train.csv')

# Columns for already located leak
cols = [
    ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'],
    ['266525925', '4b6dfc880', '2cff4bf0c', 'a3382e205', '6488c8200', '547d3135b', 'b46191036', '453128993', '2599a7eb7', '2fc60d4d9', '009319104', 'de14e7687', 'aa31dd768', '2b54cddfd', 'a67d02050', '37aab1168', '939cc02f5', '31f72667c', '6f951302c', '54723be01', '4681de4fd', '8bd53906a', '435f27009', 'f82167572', 'd428161d9', '9015ac21d', 'ec4dc7883', '22c7b00ef', 'd4cc42c3d', '1351bf96e', '1e8801477', 'b7d59d3b5', 'a459b5f7d', '580f5ff06', '39b3c553a', '1eec37deb', '692c44993', 'ce8ce671e', '88ef1d9a8', 'bf042d928'],
    ['9d5c7cb94', '197cb48af', 'ea4887e6b', 'e1d0e11b5', 'ac30af84a', 'ba4ceabc5', 'd4c1de0e2', '6d2ece683', '9c42bff81', 'cf488d633', '0e1f6696a', 'c8fdf5cbf', 'f14b57b8f', '3a62b36bd', 'aeff360c7', '64534cc93', 'e4159c59e', '429687d5a', 'c671db79e', 'd79736965', '2570e2ba9', '415094079', 'ddea5dc65', 'e43343256', '578eda8e0', 'f9847e9fe', '097c7841e', '018ab6a80', '95aea9233', '7121c40ee', '578b81a77', '96b6bd42b', '44cb9b7c4', '6192f193d', 'ba136ae3f', '8479174c2', '64dd02e44', '4ecc3f505', 'acc4a8e68', '994b946ad'],
    ['f1eeb56ae', '62ffce458', '497adaff8', 'ed1d5d137', 'faf7285a1', 'd83da5921', '0231f07ed', '7950f4c11', '051410e3d', '39e1796ab', '2e0148f29', '312832f30', '6f113540d', 'f3ee6ba3c', 'd9fc63fa1', '6a0b386ac', '5747a79a9', '64bf3a12a', 'c110ee2b7', '1bf37b3e2', 'fdd07cac1', '0872fe14d', 'ddef5ad30', '42088cf50', '3519bf4a4', 'a79b1f060', '97cc1b416', 'b2790ef54', '1a7de209c', '2a71f4027', 'f118f693a', '15e8a9331', '0c545307d', '363713112', '73e591019', '21af91e9b', '62a915028', '2ab5a56f5', 'a8ee55662', '316b978cd'],
    ['b26d16167', '930f989bf', 'ca58e6370', 'aebe1ea16', '03c589fd7', '600ea672f', '9509f66b0', '70f4f1129', 'b0095ae64', '1c62e29a7', '32a0342e2', '2fc5bfa65', '09c81e679', '49e68fdb9', '026ca57fd', 'aacffd2f4', '61483a9da', '227ff4085', '29725e10e', '5878b703c', '50a0d7f71', '0d1af7370', '7c1af7bbb', '4bf056f35', '3dd64f4c4', 'b9f75e4aa', '423058dba', '150dc0956', 'adf119b9a', 'a8110109e', '6c4f594e0', 'c44348d76', 'db027dbaf', '1fcba48d0', '8d12d44e1', '8d13d891d', '6ff9b1760', '482715cbd', 'f81c2f1dd', 'dda820122'],
    ['c928b4b74', '8e4d0fe45', '6c0e0801a', '02861e414', 'aac52d8d9', '041c5d0c9', 'd7875bb6c', 'e7c0cfd0f', 'd48c08bda', '0c9462c08', '57dd44c29', 'a93118262', '850027e38', 'db3839ab0', '27461b158', '32174174c', '9306da53f', '95742c2bf', '5831f4c76', '1e6306c7c', '06393096a', '13bdd610a', 'd7d314edc', '9a07d7b1f', '4d2671746', '822e49b95', '3c8a3ced0', '83635fb67', '1857fbccf', 'c4972742d', 'b6c0969a2', 'e78e3031b', '36a9a8479', 'e79e5f72c', '092271eb3', '74d7f2dc3', '277ef93fc', 'b30e932ba', '8f57141ec', '350473311'],
    ['06148867b', '4ec3bfda8', 'a9ca6c2f4', 'bb0408d98', '1010d7174', 'f8a437c00', '74a7b9e4a', 'cfd55f2b6', '632fed345', '518b5da24', '60a5b79e4', '3fa0b1c53', 'e769ee40d', '9f5f58e61', '83e3e2e60', '77fa93749', '3c9db4778', '42ed6824a', '761b8e0ec', 'ee7fb1067', '71f5ab59f', '177993dc6', '07df9f30c', 'b1c5346c4', '9a5cd5171', 'b5df42e10', 'c91a4f722', 'd93058147', '20a325694', 'f5e0f4a16', '5edd220bc', 'c901e7df1', 'b02dfb243', 'bca395b73', '1791b43b0', 'f04f0582d', 'e585cbf20', '03055cc36', 'd7f15a3ad', 'ccd9fc164'],
    ['df838756c', '2cb73ede7', '4dcf81d65', '61c1b7eb6', 'a9f61cf27', '1af4d24fa', 'e13b0c0aa', 'b9ba17eb6', '796c218e8', '37f57824c', 'd1e0f571b', 'f9e3b03b7', 'a3ef69ad5', 'e16a20511', '04b88be38', '99e779ee0', '9f7b782ac', '1dd7bca9f', '2eeadde2b', '6df033973', 'cdfc2b069', '031490e77', '5324862e4', '467bee277', 'a3fb07bfd', '64c6eb1cb', '8618bc1fd', '6b795a2bc', '956d228b9', '949ed0965', 'a4511cb0b', 'b64425521', '2e3c96323', '191e21b5f', 'bee629024', '1977eaf08', '5e645a169', '1d04efde3', '8675bec0b', '8337d1adc'],
    ['a1cd7b681', '9b490abb3', 'b10f15193', '05f54f417', 'a7ac690a8', 'ed6c300c2', 'd0803e3a1', 'b1bb8eac3', 'bd1c19973', 'a34f8d443', '84ec1e3db', '24018f832', '82e01a220', '4c2064b00', '0397f7c9b', 'ba42e41fa', '22d7ad48d', '9abffd22c', 'dbfa2b77f', '2c6c62b54', '9fa38def3', 'ecb354edf', '9c3154ae6', '2f26d70f4', '53102b93f', 'a36b95f78', '1fa0f78d0', '19915a6d3', 'c944a48b5', '482b04cba', '2ce77a58f', '86558e595', 'c3f400e36', '20305585c', 'f8ccfa064', 'dd771cb8e', '9aa27017e', 'cd7f0affd', '236cc1ff5', 'a3fc511cd'],
    ['920a04ee2', '93efdb50f', '15ea45005', '78c57d7cd', '91570fb11', 'c5dacc85b', '145c7b018', '590b24ab1', 'c283d4609', 'e8bd579ae', '7298ca1ef', 'ce53d1a35', 'a8f80f111', '2a9fed806', 'feb40ad9f', 'cfd255ee3', '31015eaab', '303572ae2', 'cd15bb515', 'cb5161856', 'a65b73c87', '71d64e3f7', 'ec5fb550f', '4af2493b6', '18b4fa3f5', '3d655b0ed', '5cc9b6615', '88c0ec0a6', '8722f33bb', '5ed0c24d0', '54f26ee08', '04ecdcbb3', 'ade8a5a19', 'd5efae759', 'ac7a97382', 'e1b20c3a6', 'b0fcfeab8', '438b8b599', '43782ef36', 'df69cf626'],
    ['50603ae3d', '48282f315', '090dfb7e2', '6ccaaf2d7', '1bf2dfd4a', '50b1dd40f', '1604c0735', 'e94c03517', 'f9378f7ef', '65266ad22', 'ac61229b6', 'f5723deba', '1ced7f0b4', 'b9a4f06cd', '8132d18b8', 'df28ac53d', 'ae825156f', '936dc3bc4', '5b233cf72', '95a2e29fc', '882a3da34', '2cb4d123e', '0e1921717', 'c83d6b24d', '90a2428a5', '67e6c62b9', '320931ca8', '900045349', 'bf89fac56', 'da3b0b5bb', 'f06078487', '56896bb36', 'a79522786', '71c2f04c9', '1af96abeb', '4b1a994cc', 'dee843499', '645b47cde', 'a8e15505d', 'cc9c2fc87'],
    ['b6daeae32', '3bdee45be', '3d6d38290', '5a1589f1a', '961b91fe7', '29c059dd2', 'cfc1ce276', '0a953f97e', '30b3daec2', 'fb5f5836e', 'c7525612c', '6fa35fbba', '72d34a148', 'dcc269cfe', 'bdf773176', '469630e5c', '23db7d793', 'dc10234ae', '5ac278422', '6cf7866c1', 'a39758dae', '45f6d00da', '251d1aa17', '84d9d1228', 'b98f3e0d7', '66146c12d', 'd6470c4ce', '3f4a39818', 'f16a196c6', 'b8f892930', '6f88afe65', 'ed8951a75', '371da7669', '4b9540ab3', '230a025ca', 'f8cd9ae02', 'de4e75360', '540cc3cd1', '7623d805a', 'c2dae3a5a'],
    ['d0d340214', '34d3715d5', '9c404d218', 'c624e6627', 'a1b169a3a', 'c144a70b1', 'b36a21d49', 'dfcf7c0fa', 'c63b4a070', '43ebb15de', '1f2a670dd', '3f07a4581', '0b1560062', 'e9f588de5', '65d14abf0', '9ed0e6ddb', '0b790ba3a', '9e89978e3', 'ee6264d2b', 'c86c0565e', '4de164057', '87ba924b1', '4d05e2995', '2c0babb55', 'e9375ad86', '8988e8da5', '8a1b76aaf', '724b993fd', '654dd8a3b', 'f423cf205', '3b54cc2cf', 'e04141e42', 'cacc1edae', '314396b31', '2c339d4f2', '3f8614071', '16d1d6204', '80b6e9a8b', 'a84cbdab5', '1a6d13c4a'],
    ['a9819bda9', 'ea26c7fe6', '3a89d003b', '1029d9146', '759c9e85d', '1f71b76c1', '854e37761', '56cb93fd8', '946d16369', '33e4f9a0e', '5a6a1ec1a', '4c835bd02', 'b3abb64d2', 'fe0dd1a15', 'de63b3487', 'c059f2574', 'e36687647', 'd58172aef', 'd746efbfe', 'ccf6632e6', 'f1c272f04', 'da7f4b066', '3a7771f56', '5807de036', 'b22eb2036', 'b77c707ef', 'e4e9c8cc6', 'ff3b49c1d', '800f38b6b', '9a1d8054b', '0c9b00a91', 'fe28836c3', '1f8415d03', '6a542a40a', 'd53d64307', 'e700276a2', 'bb6f50464', '988518e2d', 'f0eb7b98f', 'd7447b2c5'],
    ['87ffda550', '63c094ba4', '2e103d632', '1c71183bb', 'd5fa73ead', 'e078302ef', 'a6b6bc34a', 'f6eba969e', '0d51722ca', 'ce3d7595b', '6c5c8869c', 'dfd179071', '122c135ed', 'b4cfe861f', 'b7c931383', '44d5b820f', '4bcf15776', '51d4053c7', '1fe5d56b9', 'ea772e115', 'ad009c8b9', '68a945b18', '62fb56487', 'c10f31664', 'cbb673163', 'c8d582dd2', '8781e4b91', 'bd6da0cca', 'ca2b906e8', '11e12dbe8', 'bb0ce54e9', 'c0d2348b7', '77deffdf0', 'f97d9431e', 'a09a238d0', '935ca66a9', '9de83dc23', '861076e21', 'f02ecb19c', '166008929'],
    ['f3cf9341c', 'fa11da6df', 'd47c58fe2', '0d5215715', '555f18bd3', '134ac90df', '716e7d74d', 'c00611668', '1bf8c2597', '1f6b2bafa', '174edf08a', 'f1851d155', '5bc7ab64f', 'a61aa00b0', 'b2e82c050', '26417dec4', '53a550111', '51707c671', 'e8d9394a0', 'cbbc9c431', '6b119d8ce', 'f296082ec', 'be2e15279', '698d05d29', '38e6f8d32', '93ca30057', '7af000ac2', '1fd0a1f2a', '41bc25fef', '0df1d7b9a', '88d29cfaf', '2b2b5187e', 'bf59c51c3', 'cfe749e26', 'ad207f7bb', '11114a47a', '341daa7d1', 'a8dd5cea5', '7b672b310', 'b88e5de84'],
    ['e20edfcb8', '842415efb', '300d6c1f1', '720f83290', '069a2c70b', '87a91f998', '611151826', '74507e97f', '504e4b156', 'baa95693d', 'cb4f34014', '5239ceb39', '81e02e0fa', 'dfdf4b580', 'fc9d04cd7', 'fe5d62533', 'bb6260a44', '08d1f69ef', 'b4ced4b7a', '98d90a1d1', 'b6d206324', '6456250f1', '96f5cf98a', 'f7c8c6ad3', 'cc73678bf', '5fb85905d', 'cb71f66af', '212e51bf6', 'd318bea95', 'b70c62d47', '11d86fa6a', '3988d0c5e', '42cf36d73', '9f494676e', '1c68ee044', 'a728310c8', '612bf9b47', '105233ed9', 'c18cc7d3d', 'f08c20722'], # From this line on are new columns searched by youself, use with caution
    ['2c7e41e16', '5ad1de183', 'd8cc972fe', '5b2471c13', '841704460', '47e969ba0', 'd77583e88', '991bca4be', 'fd6e11a24', 'b5a25e7c9', '61a0acefa', '0e931cdd3', 'd9fa0485a', '614f8e1eb', '1996a153f', '289e5ecc3', 'b7d2baa45', 'ff8561ce9', '601d54a3a', '0fb0d19af', '7ee833549', '05c276b21', '0b26c77a9', 'e8387d928', '8fe3c178c', '678e2d1dd', 'd966ac62c', 'f6c436744', 'e25a65f3d', 'f1b626ac2', 'adf03173b', '57276ea06', '119230239', 'bdbb0cd24', '9282e1543', 'e7eb9a66b', '6dad99586', '4a9abd788', 'e7e41bbde', 'ebe9f985f'],
    ['a5f8c7929', '330006bce', 'b22288a77', 'de104af37', '8d81c1c27', 'd7285f250', '123ba6017', '3c6980c42', '2d3296db7', '95cdb3ab7', '05527f031', '65753f40f', '45a400659', '1d5df91e2', '233c7c17c', '2a879b4f7', 'c3c633f64', 'fdae76b2c', '05d17ab7a', 'c25078fd7', 'e209569b2', '3fd2b9645', '268b047cd', '3d350431d', '5fb9cabb1', 'b70c76dff', '3f6246360', '89e7dcacc', '12122f265', 'fcc17a41d', 'c5a742ee4', '9e711a568', '597d78667', '0186620d7', '4c095683e', '472cd130b', 'b452ba57e', '2ce2a1cdb', '50c7ea46a', '2761e2b76'],
    ['48b839509', '2b8851e90', '28f75e1a5', '0e3ef9e8f', '37ac53919', '7ca10e94b', '4b6c549b1', '467aa29ce', '74c5d55dc', '0700acbe1', '44f3640e4', 'e431708ff', '097836097', 'd1fd0b9c2', 'a0453715a', '9e3aea49a', '899dbe405', '525635722', '87a2d8324', 'faf024fa9', 'd421e03fd', '1254b628a', 'a19b05919', '34a4338bc', '08e89cc54', 'a29c9f491', 'a0a8005ca', '62ea662e7', '5fe6867a4', '8b710e161', '7ab926448', 'd04e16aed', '4e5da0e96', 'ff2c9aa8f', 'b625fe55a', '7124d86d9', '215c4d496', 'b6fa5a5fd', '55a7e0643', '0a26a3cfe'],
    ['2d60e2f7a', '11ad148bd', '54d3e247f', 'c25438f10', 'e6efe84eb', '964037597', '0196d5172', '47a8de42e', '6f460d92f', '0656586a4', '22eb11620', 'c3825b569', '6aa919e2e', '086328cc6', '9a33c5c8a', 'f9c3438ef', 'c09edaf01', '85da130e3', '2f09a1edb', '76d34bbee', '04466547a', '3b52c73f5', '1cfb3f891', '704d68890', 'f45dd927f', 'aba01a001', 'c9160c30b', '6a34d32d6', '3e3438f04', '038cca913', '504c22218', '56c679323', '002d634dc', '1938873fd', 'd37030d36', '162989a6d', 'e4dbe4822', 'ad13147bd', '4f45e06b3', 'ba480f343'],
    ['7f72c937f', '79e55ef6c', '408d86ce9', '7a1e99f69', '736513d36', '0f07e3775', 'eb5a2cc20', '2b0fc604a', 'aecd09bf5', '91de54e0a', '66891582e', '20ef8d615', '8d4d84ddc', 'dfde54714', '2be024de7', 'd19110e37', 'e637e8faf', '2d6bd8275', 'f3b4de254', '5cebca53f', 'c4255588c', '23c780950', 'bc56b26fd', '55f4891bb', '020a817ab', 'c4592ac16', '542536b93', '37fb8b375', '0a52be28f', 'bd7bea236', '1904ce2ac', '6ae9d58e0', '5b318b659', '25729656f', 'f8ee2386d', '589a5c62a', '64406f348', 'e157b2c72', '0564ff72c', '60d9fc568'],
    ['51c141e64', '0e348d340', '64e010722', '55a763d90', '13b54db14', '01fdd93d3', '1ec48dbe9', 'cf3841208', 'd208491c8', '90b0ed912', '633e0d42e', '9236f7b22', '0824edecb', '71deb9468', '1b55f7f4d', '377a76530', 'c47821260', 'bf45d326d', '69f20fee2', 'd6d63dd07', '5ab3be3e1', '93a31829f', '121d8697e', 'f308f8d9d', '0e44d3981', 'ecdef52b2', 'c69492ae6', '58939b6cc', '3132de0a3', 'a175a9aa4', '7166e3770', 'abbde281d', '23bedadb2', 'd4029c010', 'fd99222ee', 'bd16de4ba', 'fb32c00dc', '12336717c', '2ea42a33b', '50108b5b5'],
    ['7ba58c14d', '1fe02bc17', '4672a8299', '8794c72c8', 'cca45417f', '55dbd6bcb', 'e6e2c3779', '3cae817df', '973663d14', 'e8dfb33d5', '9281abeea', '11c01e052', '1520de553', 'edddb1ba5', 'c18b41ac3', '00e87edf2', 'ae72cba0a', 'eb4f2651e', '300398f1c', '6c05550b8', '9b26736c3', '24744410a', '26faf1b2e', '44f09b92d', '19975f6ff', '1bf6240eb', 'e438105db', 'cdc36a26a', '087e01c14', '828b327a6', 'cc62f0df8', '9370aa48d', 'd4815c074', '18321c252', '22fbf6997', 'feed9d437', 'f6c9661fc', '55f2b3d34', '69fe81b64', '1074273db'],
    ['4302b67ec', '75b663d7d', 'fc4a873e0', '1e9bdf471', '86875d9b0', '8f76eb6e5', '3d71c02f0', '05c9b6799', '26df61cc3', '27a7cc0ca', '9ff21281c', '3ce93a21b', '9f85ae566', '3eefaafea', 'afe8cb696', '72f9c4f40', 'be4729cb7', '8c94b6675', 'ae806420c', '63f493dba', '5374a601b', '5291be544', 'acff85649', '3690f6c26', '26c68cede', '12a00890f', 'dd84964c8', 'a208e54c7', 'fb06e8833', '7de39a7eb', '5fe3acd24', 'e53805953', '3de2a9e0d', '2954498ae', '6c3d38537', '86323e98a', 'b719c867c', '1f8a823f2', '9cc5d1d8f', 'd3fbad629'],
    ['fec5644cf', 'caa9883f6', '9437d8b64', '68811ba58', 'ef4b87773', 'ff558c2f2', '8d918c64f', '0b8e10df6', '2d6565ce2', '0fe78acfa', 'b75aa754d', '2ab9356a0', '4e86dd8f3', '348aedc21', 'd7568383a', '856856d94', '69900c0d1', '02c21443c', '5190d6dca', '20551fa5b', '79cc300c7', '8d8276242', 'da22ed2b8', '89cebceab', 'f171b61af', '3a07a8939', '129fe0263', 'e5b2d137a', 'aa7223176', '5ac7e84c4', '9bd66acf6', '4c938629c', 'e62c5ac64', '57535b55a', 'a1a0084e3', '2a3763e18', '474a9ec54', '0741f3757', '4fe8b17c2', 'd5754aa08'],
    ['6cf7644e0', 'c0004231c', '25968f96e', 'fb8395d97', 'ee9e70298', 'e662f1672', 'eda856f5f', '49f11e712', '43dc0f90c', '11d9e8383', '0738fe5ff', '3268914c7', '08d17e384', '1a67a2101', '406a11b5a', 'e2e6f1d23', '2862eec4f', '9c6bf2983', '0a4e510ee', '14d2c6d95', '0a03426de', '82ade3db6', '8909b68e1', 'aaed60a3d', '879e1f51a', '5cbf1b3ea', '1ecddbaa0', 'b96718230', 'bd550871c', 'c55eb4392', '0743b9c08', '5580c77b0', '25613ca0a', 'de66047b0', '2322dbbbb', '760ef38f1', 'da5c36eee', '2de811f82', 'a760e7f65', '5d8a2a27d'],
    ['9fa984817', '3d23e8abd', '1b681c3f0', '3be4dad48', 'dcfcddf16', 'b25319cb3', 'b14026520', 'c5cb7200e', 'ede70bfea', 'e5ddadc85', '07cb6041d', 'df6a71cc7', 'dc60842fb', '3a90540ab', '6bab7997a', 'c87f4fbfb', '21e0e6ae3', '9b39b02c0', '5f5cfc3c0', '35da68abb', 'f0aa40974', '625525b5d', 'd7978c11c', '2bbcbf526', 'bc2bf3bcd', '169f6dda5', '4ceef6dbd', '9581ec522', 'd4e8dd865', 'bf8150471', '542f770e5', 'b05eae352', '3c209d9b6', 'b2e1308ae', '786351d97', 'e5a8e9154', '2b85882ad', 'dc07f7e11', '14c2463ff', '14a5969a6'],
    ['0f8d7b98e', 'c30ff7f31', 'ac0e2ebd0', '24b2da056', 'bd308fe52', '476d95ef1', '202acf9bd', 'dbc0c19ec', '06be6c2bb', 'd8296080a', 'f977e99dc', '2191d0a24', '7db1be063', '1bc285a83', '9a3a1d59b', 'c4d657c5b', 'a029667de', '21bd61954', '16bf5a9a2', '0e0f8504b', '5910a3154', 'ba852cc7a', '685059fcd', '21d6a4979', '78947b2ad', '1435ecf6b', '3839f8553', 'e9b5b8919', 'fa1dd6e8c', '632586103', 'f016fd549', 'c25ea08ba', '7da54106c', 'b612f9b7e', 'e7c0a50e8', '29181e29a', '395dbfdac', '1beb0ce65', '04dc93c58', '733b3dc47'],
    ['ccc7609f4', 'ca7ea80a3', 'e509be270', '3b8114ab0', 'a355497ac', '27998d0f4', 'fa05fd36e', '81aafdb57', '4e22de94f', 'f0d5ffe06', '9af753e9d', 'f1b6cc03f', '567d2715c', '857020d0f', '99fe351ec', '3e5dab1e3', '001476ffa', '5a5eabaa7', 'cb5587baa', '32cab3140', '313237030', '0f6386200', 'b961b0d59', '9452f2c5f', 'bcfb439ee', '04a22f489', '7e58426a4', 'a4c9ea341', 'ffdc4bcf8', '1a6d866d7', 'd7334935b', '298db341e', '08984f627', '8367dfc36', '5d9f43278', '7e3e026f8', '37c10d610', '5a88b7f01', '324e49f36', '99f466457'],
    ['86cefbcc0', '717eff45b', '7d287013b', '8d7bfb911', 'aecaa2bc9', '193a81dce', '8dc7f1eb9', 'c5a83ecbc', '60307ab41', '3da5e42a7', 'd8c61553b', '072ac3897', '1a382b105', 'f3a4246a1', '4e06e4849', '962424dd3', 'a3da2277a', '0a69cc2be', '408d191b3', '98082c8ef', '96b66294d', 'cc93bdf83', 'ffa6b80e2', '226e2b8ac', '678b3f377', 'b56f52246', '4fa02e1a8', '2ef57c650', '9aeec78c5', '1477c751e', 'a3c187bb0', '1ce516986', '080cd72ff', '7a12cc314', 'ead538d94', '480e78cb0', '737d43535', 'a960611d7', '4416cd92c', 'd5e6c18b0'],
    ['2135fa05a', 'e8a3423d6', '90a438099', '7ad6b38bd', '60e45b5ee', '2b9b1b4e2', 'd6c82cd68', '923114217', 'b361f589e', '04be96845', 'ee0b53f05', '21467a773', '47665e3ce', 'a6229abfb', '9666bfe76', '7dcc40cda', '17be6c4e7', 'a89ab46bb', '9653c119c', 'cc01687d0', '60e9cc05b', 'ffcec956f', '51c250e53', '7344de401', 'a15b2f707', 'a8e607456', 'dbb8e3055', '2a933bcb8', 'b77bc4dac', '58d9f565a', '17068424d', '7453eb289', '027a2206a', '343042ed9', 'c8fb3c2d8', '29eddc376', '1c873e4a6', '588106548', '282cfe2ad', '358dc07d0'],
    ['a3e023f65', '9126049d8', '6eaea198c', '5244415dd', '0616154cc', '2165c4b94', 'fc436be29', '1834f29f5', '9d5af277d', 'c6850e7db', '6b241d083', '56f619761', '45319105a', 'fcda960ae', '07746dcda', 'c906cd268', 'c24ea6548', '829fb34b8', '89ebc1b76', '22c019a2e', '1e16f11f3', '94072d7a3', '59dfc16da', '9886b4d22', '0b1741a7f', 'a682ef110', 'e26299c3a', '5c220a143', 'ac0493670', '8d8bffbae', '68c7cf320', '3cea34020', 'e9a8d043d', 'afb6b8217', '5780e6ffa', '26628e8d8', '1de4d7d62', '4c53b206e', '99cc87fd7', '593cccdab'],
    ['8f6514df0', '6679fe54f', '5e62457b7', 'f17ff4efd', 'ec7f7017f', 'c02ab7d25', '8c309c553', 'e0b968d7b', '22b980fc8', '3b6b46221', '3e4a6796d', 'c680e9350', '834fb292d', 'e3d33877c', '4052a9419', 'b95be4138', '16517c8b0', '219e051b5', 'a6fbe0987', '37d7af8ad', 'b84b2f72d', '775577e6f', '4f0c5f900', 'a68b83290', '2a2832b07', 'ce1f5b02a', 'a6c9347a7', '82c9b4fcd', '7f78a36f7', 'f49ff3269', '89cffafe9', 'aeb3a6ccf', 'c7753cbfc', '4d6a1439e', '2123a4f36', '5c56fccf1', '03bfe48b2', '6beb0b35d', '9fb38aabe', 'ae141696e'],
    ['0d7692145', '62071f7bc', 'ab515bdeb', 'c30c6c467', 'eab76d815', 'b6ee6dae6', '49063a8ed', '4cb2946ce', '6c27de664', '772288e75', 'afd87035a', '44f2f419e', '754ace754', 'e803a2db0', 'c70f77ef2', '65119177e', '3a66c353a', '4c7768bff', '9e4765450', '24141fd90', 'dc8b7d0a8', 'ba499c6d9', '8b1379b36', '5a3e3608f', '3be3c049e', 'a0a3c0f1b', '4d2ca4d52', '457bd191d', '6620268ab', '9ad654461', '1a1962b67', '7f55b577c', '989d6e0f5', 'bc937f79a', 'e059a8594', '3b74ac37b', '555265925', 'aa37f9855', '32c8b9100', 'e71a0278c'],
    ['63be1f619', '36a56d23e', '9e2040e5b', 'a00a63886', '4edc3388d', '5f11fbe33', '26e998afd', 'f7faf2d9f', '992b5c34d', 'f7f553aea', '7e1c4f651', 'f5538ee5c', '711c20509', '55338de22', '374b83757', 'f41f0eb2f', 'bf10af17e', 'e2979b858', 'd3ed79990', 'fe0c81eff', '5c0df6ac5', '82775fc92', 'f1c20e3ef', 'fa9d6b9e5', 'a8b590c6e', 'b5c4708ad', 'c9aaf844f', 'fe3fe2667', '50a6c6789', '8761d9bb0', 'b6403de0b', '2b6f74f09', '5755fe831', '91ace30bd', '84067cfe0', '15e4e8ee5', 'd01cc5805', '870e70063', '2bd16b689', '8895ea516'],
    ['5b465f819', 'a2aa0e4e9', '944e05d50', '4f8b27b6b', 'a498f253f', 'c73c31769', '025dea3b3', '616c01612', 'f3316966c', '83ea288de', '2dbeac1de', '47b7b878f', 'b4d41b335', '686d60d8a', '6dcd9e752', '7210546b2', '78edb3f13', '7f9d59cb3', '30992dccd', '26144d11f', 'a970277f9', '0aea1fd67', 'dc528471e', 'd51d10e38', 'efa99ed98', '48420ad48', '7f38dafa6', '1af4ab267', '3a13ed79a', '73445227e', '971631b2d', '57c4c03f6', '7f91dc936', '0784536d6', 'c3c3f66ff', '052a76b0f', 'ffb34b926', '9d4f88c7b', '442b180b6', '948e00a8d'],
    ['c13ee1dc9', 'abb30bd35', 'd2919256b', '66728cc11', 'eab8abf7a', 'cc03b5217', '317ee395d', '38a92f707', '467c54d35', 'e8f065c9d', '2ac62cba5', '6495d8c77', '94cdda53f', '13f2607e4', '1c047a8ce', '28a5ad41a', '05cc08c11', 'b0cdc345e', '38f49406e', '773180cf6', '1906a5c7e', 'c104aeb2e', '8e028d2d2', '0dc333fa1', '28a785c08', '03ee30b8e', '8e5a41c43', '67102168f', '8b5c0fb4e', '14a22ab1a', '9fc776466', '4aafb7383', '8e1dfcb94', '55741d46d', '8f940cb1b', '758a9ab0e', 'fd812d7e0', '4ea447064', '6562e2a2c', '343922109'],
    ['9d4428628', '37f11de5d', '39549da61', 'ceba761ec', '4c60b70b8', '304ebcdbc', '823ac378c', '4e21c4881', '5ee81cb6e', 'eb4a20186', 'f6bdb908a', '6654ce6d8', '65aa7f194', '00f844fea', 'c4de134af', 'a240f6da7', '168c50797', '13d6a844f', '7acae7ae9', '8c61bede6', '45293f374', 'feeb05b3f', 'a5c62af4a', '22abeffb6', '1d0aaa90f', 'c46028c0f', '337b3e53b', 'd6af4ee1a', 'cde3e280a', 'c83fc48f2', 'f99a09543', '85ef8a837', 'a31ba11e6', '64cabb6e7', '93521d470', '46c525541', 'cef9ab060', '375c6080e', '3c4df440f', 'e613715cc']    
]

In [24]:
# Columns for leak padded with 1166666.66
pattern_116 = pd.read_csv('./new_leak_patterns/pattern_1166666.66.csv')
new_cols = list(pattern_116.columns)[1:-1]
row_index = list(pattern_116['Unnamed: 0'])
# the preceeding column before columns in file 'pattern_1166666.66.csv' is added
cols.append(['c928b4b74']+new_cols[:11]+['850027e38']+new_cols[12:])

In [25]:
# Columns for leak padded with 1964666.66
pattern_196 = pd.read_csv('./new_leak_patterns/pattern_1964666.66.csv')
new_cols = list(pattern_196.columns)[1:-1]
# the preceeding column before columns in file 'pattern_1964666.66.csv' is added
cols.append(['ced6a7e91']+new_cols)

In [26]:
# Columns for leak padded with 2002166.66
pattern_200 = pd.read_csv('./new_leak_patterns/pattern_2002166.66.csv')
new_cols = list(pattern_200.columns)[1:-1]
# the preceeding column before columns in file 'pattern_2002166.66.csv' is added
cols.append(['5030aed26']+new_cols)

In [27]:
# Columns for leak padded with 3160000
pattern_316 = pd.read_csv('./new_leak_patterns/pattern_3160000.csv')
new_cols = list(pattern_316.columns)[1:-1]
# the preceeding column before columns in file 'pattern_3160000.csv' is added
cols.append(['81de0d45e']+new_cols)

In [28]:
# Columns for leak padded with 3255483.88
pattern_325 = pd.read_csv('./new_leak_patterns/pattern_3255483.88.csv')
new_cols = list(pattern_325.columns)[1:-1]
# the preceeding column before columns in file 'pattern_3255483.88.csv' is added
cols.append(['1d9078f84']+new_cols)

In [29]:
# Columns for leak padded with 812666.66
pattern_812 = pd.read_csv('./new_leak_patterns/pattern_812666.66.csv')
new_cols = list(pattern_812.columns)[1:-1]
# the preceeding column before columns in file 'pattern_812666.66.csv' is added
cols.append(['1ad24da13']+new_cols)

Since lots of leak columns are already located. It's possible to determine some rows in train data are one time stamp later than other rows. Then if one column is before another in the new leak we are trying to find, values in these earlier rows of the first column should be almost equal to values in later rows of the second column.

In [30]:
# cols_1 stores all columns except for the last one in each leak group
# cols_2 stores all columns except for the first one in each leak group
cols_1 = []
cols_2 = []
for i in np.arange(len(cols)):
    cols_1.extend(cols[i][:-1])
    cols_2.extend(cols[i][1:])
# trash_key = tuple([0]*(len(new_cols)-2-lag))
d1 = train[cols_1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
d1['item_index'] = d1.index
d2 = train[cols_2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
d2['matched_index'] = d2.index
# d2['nonzero'] = df[cols_2].apply(lambda x: sum(x != 0),axis=1)
# d2 = d2[d2['nonzero'] > 0]
d3 = d2[~d2.duplicated(['key'], keep=False)]
# d3 = d3[d3['key'] != trash_key]

d4 = d1.merge(d3, how='left', on='key')
# row_match is a list that contains two lists of row indexs. Rows in the first list are one time stamp earlier than rows in the second list
row_match =[list(d4.loc[d4['matched_index'].notna(),'item_index']), list(d4.loc[d4['matched_index'].notna(),'matched_index'])]

In [31]:
# Exclude columns that contain only zeros in all rows of row_match[0], since they will certainly match each other
empty_columns = []
transact_cols = [f for f in train.columns if f not in ['ID','target']]
for column in transact_cols:
    if sum(train.loc[row_match[0],column] != 0) == 0:
        empty_columns.append(column)

In [32]:
found_columns = [] # found_columns will store all columns that is already examined and empty columns
for i in np.arange(len(cols)):
    found_columns.extend(cols[i])

found_columns.extend(empty_columns)
print('Number of leak columns already found and empty columns: {}'.format(len(found_columns)))

Number of leak columns already found and empty columns: 2292


In [33]:
leak_groups = [] # Stores all groups of leak columns
cur_group = [] # Stores current leak group that is still being extended
count = len(found_columns)
for first_column in transact_cols:
    if first_column not in found_columns:
        if len(found_columns)-count > 100:
            print("search on {} columns is finished".format(len(found_columns)))
            count = len(found_columns)
        cur_group = [first_column]
        found_columns.append(first_column)
        last_column = first_column
        match_found = True
        while(match_found):
            match_found = False
            for next_column in transact_cols:
                if next_column not in found_columns:
                    # Search for a column that can be appended to the tail of cur_group
                    if tuple(train.loc[row_match[0],cur_group[-1]]) == tuple(train.loc[row_match[1],next_column]):
                        cur_group.append(next_column)
                        found_columns.append(next_column)
                        match_found = True
                        break
                    # Search for a column that can be added to the head of cur_group
                    if tuple(train.loc[row_match[1],cur_group[0]]) == tuple(train.loc[row_match[0],next_column]):
                        cur_group.insert(0,next_column)
                        found_columns.append(next_column)
                        match_found = True
                        break
        leak_groups.append(cur_group)

search on 2432 columns is finished
search on 2565 columns is finished
search on 2685 columns is finished
search on 2812 columns is finished
search on 2947 columns is finished
search on 3053 columns is finished
search on 3173 columns is finished
search on 3300 columns is finished
search on 3425 columns is finished
search on 3545 columns is finished
search on 3685 columns is finished
search on 3792 columns is finished
search on 3893 columns is finished
search on 4020 columns is finished
search on 4160 columns is finished
search on 4265 columns is finished
search on 4375 columns is finished
search on 4500 columns is finished
search on 4606 columns is finished
search on 4745 columns is finished
search on 4847 columns is finished
search on 4949 columns is finished


In [34]:
# Pick out leak groups that have 40 columns
useful_leak_groups = [f for f in leak_groups if len(f) == 40]

In [36]:
print('Number of new leak groups found: {}'.format(len(useful_leak_groups)))

Number of new leak groups found: 50


In [35]:
# Exam one of leak groups that have 40 columns
m = 23
train.loc[row_index,useful_leak_groups[m]]

,c95423453,84d4d30b8,3770cb9fa,3bb7bc789,13a2ecd25,5c20afdb3,29bf806d6,5c0b5d1d4,4e98771c9,3974799dd,04ef53271,57412a852,d5d85bc77,963c9c0ac,a48a740ef,dacebaeaf,174bec4d1,4f0b30912,e8d16b5b5,b728093e6,b4a4a4df8,44c06f79a,7f3479656,ee7e4581d,890d30d93,59d2470ed,f18d3931b,055232767,366841793,211314d56,538df95cd,dc6902c31,8ca717e6d,15b0fe826,c6cbb2938,87ba106d3,4a9e09bff,f7b2550f2,133714358,16be01500
1757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3809,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
511,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3303,426000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4095,0,426000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,426000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4209,0,0,0,0,426000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1696,0,0,0,0,0,426000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
